Start jupyter-lab

```bash
jupyter-lab --notebook-dir=$HOME/gits/gerashegalov/rapids-shell/src/jupyter
```
and/or connect to the URL VS Code with Jupyter extension

In [ ]:
%%init_spark

import os
import sys
import tempfile

# Environment
home_dir = os.environ['HOME']
work_dir = f"{home_dir}/jupyter_run_dir"
rapids_version = '22.12.0-SNAPSHOT'
locally_built = True
tz = 'UTC'
os.environ['SPARK_HOME'] = f"{home_dir}/dist/spark-3.1.1-bin-hadoop3.2"
os.environ['TZ'] = tz
sys.path.append(f"{home_dir}/gits/NVIDIA/spark-rapids/integration_tests/src/main/python")

# Spark Master
cores_per_exec = 1
spark_master = f"local-cluster[1,{cores_per_exec},4096]"
# spark_master = f"local[{cores_per_exec}]"

# debugger string
# jdwp = '-agentlib:jdwp=transport=dt_socket,server=n,address=localhost:5005'
jdwp = ''

if locally_built:
  launcher.jars = [
    f"{home_dir}/gits/NVIDIA/spark-rapids/dist/target/rapids-4-spark_2.12-{rapids_version}-cuda11.jar"
  ]
else:
  launcher.packages = [
    f"com.nvidia:rapids-4-spark_2.12:{rapids_version}"
  ]

launcher.conf.spark.driver.extraJavaOptions = f"\"-Dai.rapids.refcount.debug=true -Dlog4j.debug=true {jdwp}\""
launcher.conf.spark.executor.extraJavaOptions = "\"-Dai.rapids.refcount.debug=true -Dlog4j.debug=true\""
launcher.conf.spark.executorEnv.TZ = tz
launcher.conf.spark.master = spark_master
launcher.conf.spark.plugins = "com.nvidia.spark.SQLPlugin"
launcher.conf.set('spark.rapids.sql.explain', 'ALL')

In [ ]:
spark

In [ ]:
%%python
import os
print(f"Driver Python version: {sc.pythonVer}\n")
print(f"Worker Python version: {os.environ['PYSPARK_PYTHON']}\n")

## Datagen from integration tests

In [ ]:
%%python
import spark_init_internal
setattr(spark_init_internal, '_spark', spark)
from data_gen import *

In [ ]:
%%python
# datagen works?
df = unary_op_df(spark=spark, gen=TimestampGen(nullable=False), length=3, num_slices=1)
df.createOrReplaceTempView('DFT')
df.show(truncate=False)

## Work in Scala by default

In [ ]:
val scalaDF = spark.sql("SELECT * FROM DFT")
scalaDF.selectExpr("MAX(a)").collect()